In [1]:
# First version of the forecast pipeline program
# Author: Rong Lin
# Date: 2024/08/08

# please use py39env on RONG's windows laptop.
import pandas as pd
import numpy as np
import datetime
import pfsspy

# C:\Users\87488\AppData\Local\Temp\ipykernel_3384\1689277766.py:6: DeprecationWarning: 
# Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
# (to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
# but was not found to be installed on your system.
# If this would cause problems for you,
# please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466

# solution: pip install pyarrow



C:\Users\87488\.conda\envs\py39env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
pfsspy.__version__

'1.2.0'

In [3]:
import sunpy
sunpy.__version__

'5.1.2'

In [4]:
# input: epoches to be forecast
# TODO: make the slide buttton...
date_dict = {'date':[datetime.datetime(2021, 6, 11, 14),
                     datetime.datetime(2021, 6, 11, 15),
                     datetime.datetime(2021, 6, 12, 15),
                     datetime.datetime(2021, 6, 12, 16),
                     datetime.datetime(2021, 6, 12, 17),
                     datetime.datetime(2021, 6, 13, 16)]}

# calculate the date for 3, 4, 5 and 6 days ago, for each epoch
date_df = pd.DataFrame(date_dict)
date_df['3_days_ago'] = date_df['date'] + datetime.timedelta(days=-3)
date_df['4_days_ago'] = date_df['date'] + datetime.timedelta(days=-4)
date_df['5_days_ago'] = date_df['date'] + datetime.timedelta(days=-5)
date_df['6_days_ago'] = date_df['date'] + datetime.timedelta(days=-6)
print(date_df)

                 date          3_days_ago          4_days_ago  \
0 2021-06-11 14:00:00 2021-06-08 14:00:00 2021-06-07 14:00:00   
1 2021-06-11 15:00:00 2021-06-08 15:00:00 2021-06-07 15:00:00   
2 2021-06-12 15:00:00 2021-06-09 15:00:00 2021-06-08 15:00:00   
3 2021-06-12 16:00:00 2021-06-09 16:00:00 2021-06-08 16:00:00   
4 2021-06-12 17:00:00 2021-06-09 17:00:00 2021-06-08 17:00:00   
5 2021-06-13 16:00:00 2021-06-10 16:00:00 2021-06-09 16:00:00   

           5_days_ago          6_days_ago  
0 2021-06-06 14:00:00 2021-06-05 14:00:00  
1 2021-06-06 15:00:00 2021-06-05 15:00:00  
2 2021-06-07 15:00:00 2021-06-06 15:00:00  
3 2021-06-07 16:00:00 2021-06-06 16:00:00  
4 2021-06-07 17:00:00 2021-06-06 17:00:00  
5 2021-06-08 16:00:00 2021-06-07 16:00:00  


In [5]:
# merge all dates from 3, 4, 5, and 6 days ago,
# so that we don't need to download for multiple times
melted_df = pd.melt(date_df, value_vars=['3_days_ago', '4_days_ago', '5_days_ago', '6_days_ago'], value_name='uniq_date')
unique_dates = pd.to_datetime(melted_df['uniq_date']).drop_duplicates()
unique_sorted_dates = unique_dates.sort_values().reset_index(drop=True)
unique_sorted_dates = pd.DataFrame(unique_sorted_dates)
unique_sorted_dates['path'] =''
print(unique_sorted_dates)

             uniq_date path
0  2021-06-05 14:00:00     
1  2021-06-05 15:00:00     
2  2021-06-06 14:00:00     
3  2021-06-06 15:00:00     
4  2021-06-06 16:00:00     
5  2021-06-06 17:00:00     
6  2021-06-07 14:00:00     
7  2021-06-07 15:00:00     
8  2021-06-07 16:00:00     
9  2021-06-07 17:00:00     
10 2021-06-08 14:00:00     
11 2021-06-08 15:00:00     
12 2021-06-08 16:00:00     
13 2021-06-08 17:00:00     
14 2021-06-09 15:00:00     
15 2021-06-09 16:00:00     
16 2021-06-09 17:00:00     
17 2021-06-10 16:00:00     


In [6]:
# DOWNLOADING dependencies: modules, constants and functions
import os, gzip
import requests
from bs4 import BeautifulSoup
    
URL_HEADER = 'https://gong2.nso.edu/oQR/zqs/'
FOLDER = 'gong_data/'

def find_nearest_file(time_filename_dict, input_datetime, time_diff=0.5):
    # time diff is in hour(s)
    # the dict: key is time
    # find corresponding value in dict, with the key-time nearest to the input_datetime
    if not time_filename_dict:
        return None
    nearest_dt = min(time_filename_dict.keys(), key=lambda dt: abs(dt - input_datetime))
    if abs(nearest_dt - input_datetime) > datetime.timedelta(hours=time_diff):
        return None
    else:
        return time_filename_dict[nearest_dt]
        
def get_url_of_downloading_page(date_inp):
    year2_month_day_str = str(date_inp.year)[-2:] + '%02d' % date_inp.month + '%02d' % date_inp.day
    url = URL_HEADER + year4_month_str + '/mrzqs' + year2_month_day_str + '/'
    return url
    
def get_downloading_page_dict(url,date_inp):
    # analyse the downlaoding page, and get the filename list with corresponding file url
    r0 = requests.get(url)
    soup0 = BeautifulSoup(r0.content)
    links0 = soup0.findAll('a')
    # build dict for that page
    res = dict()
    for link0 in links0:
        if link0['href'].startswith('mrzqs'):
            url1 = url + link0['href']
            splitted_href = link0['href'].split('t')
            hours_str = splitted_href[1][:2]
            mins_str = splitted_href[1][2:4]
            file_time = datetime.datetime(date_inp.year,date_inp.month,date_inp.day,int(hours_str),int(mins_str))
            res[file_time] = url1
    return res    

In [7]:
# DOWNLOADING required files
year_month_day_previous_i = None
for i_date in range(len(unique_sorted_dates)):
    date_input = unique_sorted_dates['uniq_date'][i_date]
    date_input = date_input.to_pydatetime()
    year4_month_str = str(date_input.year) + '%02d' % date_input.month
    year2_month_day_str = str(date_input.year)[-2:]+'%02d'%date_input.month+'%02d'%date_input.day
    if (year_month_day_previous_i is not None) and (year2_month_day_str == year_month_day_previous_i):
        pass
        # still use the "time_filename_dict_this_day"
    else:
        # get url of downloading page
        url0 = get_url_of_downloading_page(date_input)
        
        time_filename_dict_this_day = get_downloading_page_dict(url0,date_input)   
        
        # analyse the downlaoding page, and get the filename list with corresponding file url
        r0 = requests.get(url0)
        soup0 = BeautifulSoup(r0.content)
        links0 = soup0.findAll('a')

        # build dict for that page
        time_filename_dict_this_day = dict()
        for link0 in links0:
            if link0['href'].startswith('mrzqs'):
                url1 = url0 + link0['href']
                splitted_href = link0['href'].split('t')
                hours_str = splitted_href[1][:2]
                mins_str = splitted_href[1][2:4]
                file_time = datetime.datetime(date_input.year,date_input.month,date_input.day,int(hours_str),int(mins_str))
                time_filename_dict_this_day[file_time] = url1
        year_month_day_previous_i = year2_month_day_str        
    
    gz_fileurl = find_nearest_file(time_filename_dict_this_day, date_input)
    if gz_fileurl is None:
        gz_filename = None
        path_file_full = None
    else:
        gz_filename = gz_fileurl.split('/')[-1] # NoneType does not have it
        path = FOLDER + year4_month_str + '/' + gz_filename[:11]+'/'
        if not os.path.exists(path):
            os.makedirs(path)
        path_file = path + gz_filename
        if not os.path.exists(path_file):
            r2 = requests.get(gz_fileurl)
            data = r2.content
            with open(path_file, 'wb') as f:
                f.write(data)
            path_file_full = path_file.replace('.gz', '')
            file_data = gzip.GzipFile(path_file)
            with open(path_file_full, "wb+") as pfu:
                pfu.write(file_data.read())
            file_data.close()
        else:
            path_file_full = path_file.replace('.gz', '')
            pass
    # print(date_input, year2_month_day_str, url0, gz_filename, path_file_full)
    unique_sorted_dates.loc[i_date,'path'] = path_file_full

In [8]:
# arrange dataframe
date_df = date_df.merge(unique_sorted_dates, left_on='3_days_ago', right_on='uniq_date',
                        how='left').drop(columns=['uniq_date']).rename(columns={'path': 'path_3_days_ago'})
date_df = date_df.merge(unique_sorted_dates, left_on='4_days_ago', right_on='uniq_date',
                        how='left').drop(columns=['uniq_date']).rename(columns={'path': 'path_4_days_ago'})
date_df = date_df.merge(unique_sorted_dates, left_on='5_days_ago', right_on='uniq_date',
                        how='left').drop(columns=['uniq_date']).rename(columns={'path': 'path_5_days_ago'})
date_df = date_df.merge(unique_sorted_dates, left_on='6_days_ago', right_on='uniq_date',
                        how='left').drop(columns=['uniq_date']).rename(columns={'path': 'path_6_days_ago'})

conditions = date_df[['path_3_days_ago', 'path_4_days_ago', 'path_5_days_ago', 'path_6_days_ago']].notnull().all(axis=1)
date_df['all_paths_present'] = conditions
print(date_df.head)

<bound method NDFrame.head of                  date          3_days_ago          4_days_ago  \
0 2021-06-11 14:00:00 2021-06-08 14:00:00 2021-06-07 14:00:00   
1 2021-06-11 15:00:00 2021-06-08 15:00:00 2021-06-07 15:00:00   
2 2021-06-12 15:00:00 2021-06-09 15:00:00 2021-06-08 15:00:00   
3 2021-06-12 16:00:00 2021-06-09 16:00:00 2021-06-08 16:00:00   
4 2021-06-12 17:00:00 2021-06-09 17:00:00 2021-06-08 17:00:00   
5 2021-06-13 16:00:00 2021-06-10 16:00:00 2021-06-09 16:00:00   

           5_days_ago          6_days_ago  \
0 2021-06-06 14:00:00 2021-06-05 14:00:00   
1 2021-06-06 15:00:00 2021-06-05 15:00:00   
2 2021-06-07 15:00:00 2021-06-06 15:00:00   
3 2021-06-07 16:00:00 2021-06-06 16:00:00   
4 2021-06-07 17:00:00 2021-06-06 17:00:00   
5 2021-06-08 16:00:00 2021-06-07 16:00:00   

                                     path_3_days_ago  \
0                                               None   
1  gong_data/202106/mrzqs210608/mrzqs210608t1514c...   
2  gong_data/202106/mrzqs21060

In [9]:
# GONG, PFSS and torch dependencies: modules, constants and functions
import sunpy.map
import pfsspy
from sunpy.sun import constants
import astropy.units as u

import torch
print(f"Torch version: {torch.__version__}")
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

nrho = 30
rss = 2.5

def get_gong_map_obj(path_to_file):
    # create gong map object
    gmap_temp = sunpy.map.Map(path_to_file)
    gmap = sunpy.map.Map(gmap_temp.data - np.mean(gmap_temp.data), gmap_temp.meta)

    # added by RONG 2024/8/15, to prevent some 
    gmap.meta['rsun_ref'] = constants.radius / u.m
    gmap.meta['bunit'] = 'G'
    return gmap

def get_ss_graphs(df, i_df):
    col = df.iloc[i_df]
    
    gong_map_3 = get_gong_map_obj(col['path_3_days_ago'])
    input_3 = pfsspy.Input(gong_map_3, nrho, rss)
    output_3 = pfsspy.pfss(input_3)
    graph_3 = output_3.source_surface_br.data
    lat_3 = int(gong_map_3.carrington_latitude.to_value().round())
    sub_graph_3 = graph_3[lat_3+90-45:lat_3+90+45,90:-90]
    
    gong_map_4 = get_gong_map_obj(col['path_4_days_ago'])
    input_4 = pfsspy.Input(gong_map_4, nrho, rss)
    output_4 = pfsspy.pfss(input_4)
    graph_4 = output_4.source_surface_br.data
    lat_4 = int(gong_map_4.carrington_latitude.to_value().round())
    sub_graph_4 = graph_4[lat_4+90-45:lat_4+90+45,90:-90]
    
    gong_map_5 = get_gong_map_obj(col['path_5_days_ago'])
    input_5 = pfsspy.Input(gong_map_5, nrho, rss)
    output_5 = pfsspy.pfss(input_5)
    graph_5 = output_5.source_surface_br.data
    lat_5 = int(gong_map_5.carrington_latitude.to_value().round())
    sub_graph_5 = graph_5[lat_5+90-45:lat_5+90+45,90:-90]
    
    gong_map_6 = get_gong_map_obj(col['path_6_days_ago'])
    input_6 = pfsspy.Input(gong_map_6, nrho, rss)
    output_6 = pfsspy.pfss(input_6)
    graph_6 = output_6.source_surface_br.data
    lat_6 = int(gong_map_6.carrington_latitude.to_value().round())
    sub_graph_6 = graph_6[lat_6+90-45:lat_6+90+45,90:-90]

    ss_graphs = np.array([sub_graph_6,sub_graph_5,sub_graph_4,sub_graph_3])
    
    return ss_graphs

Torch version: 2.4.0


In [10]:
%%time
can_forecast_indices = date_df.index[date_df['all_paths_present'] == True].tolist()
can_forecast_indices

input_for_model = list()
for i in can_forecast_indices:
    input_for_model.append(get_ss_graphs(date_df,i))
    
input_for_model = np.array(input_for_model)
input_for_model = torch.from_numpy(input_for_model)
input_for_model.shape

CPU times: total: 3.5 s
Wall time: 36.6 s


torch.Size([5, 4, 90, 180])

In [11]:
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(
            in_channels=4, out_channels=16, kernel_size=(10,10), stride=1, padding=5
        )
        self.conv1_bn = nn.BatchNorm2d(16)
        self.conv2 = nn.Conv2d(
            in_channels=16, out_channels=32, kernel_size=(3, 3), stride=1, padding=1
        )
        self.conv2_bn = nn.BatchNorm2d(32)
        self.conv3 = nn.Conv2d(
            in_channels=32, out_channels=64, kernel_size=(3, 3), stride=1, padding=1
        )
        self.conv3_bn = nn.BatchNorm2d(64)
        self.dropout1 = nn.Dropout(p=0.3)
        self.dropout2 = nn.Dropout(p=0.4)
        self.fc1 = nn.Linear(64 * 50, 64)
        self.fc2 = nn.Linear(64, 1)
        self.pool = nn.MaxPool2d(kernel_size=3, stride=3)
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)

    def forward(self, x):
        x = self.conv1(x)
        x = self.conv1_bn(x)
        x = F.relu(x)
        x = self.pool(x)
        x = self.conv2(x)
        x = self.conv2_bn(x)
        x = F.relu(x)
        x = self.pool(x)
        x = self.conv3(x)
        x = self.conv3_bn(x)
        x = F.relu(x)
        x = self.pool2(x)
        x = torch.flatten(x, 1)
        x = self.dropout1(x)
        x = self.fc1(x)
        x = self.dropout2(x)
        x = F.relu(x)
        x = self.fc2(x)
        x = torch.sigmoid(x)
        x = x * 1200.0
        output = x
        return output

In [12]:
# MODEL_1 = torch.load("./Models/" + "20230613MODEL_temporary_min_validation_loss1.pt")
# MODEL_2 = torch.load("./Models/" + "20230613MODEL_temporary_min_validation_loss2.pt")
# MODEL_3 = torch.load("./Models/" + "20230613MODEL_temporary_min_validation_loss3.pt")
# MODEL_4 = torch.load("./Models/" + "20230613MODEL_temporary_min_validation_loss4.pt")
# MODEL_5 = torch.load("./Models/" + "20230613MODEL_temporary_min_validation_loss5.pt")
# MODEL_6 = torch.load("./Models/" + "20230613MODEL_temporary_min_validation_loss6.pt")
# MODEL_7 = torch.load("./Models/" + "20230613MODEL_temporary_min_validation_loss7.pt")

# torch.save(MODEL_1.state_dict(), "./Models/published_model_1.pt")
# torch.save(MODEL_2.state_dict(), "./Models/published_model_2.pt")
# torch.save(MODEL_3.state_dict(), "./Models/published_model_3.pt")
# torch.save(MODEL_4.state_dict(), "./Models/published_model_4.pt")
# torch.save(MODEL_5.state_dict(), "./Models/published_model_5.pt")
# torch.save(MODEL_6.state_dict(), "./Models/published_model_6.pt")
# torch.save(MODEL_7.state_dict(), "./Models/published_model_7.pt")

In [13]:
MODEL_1 = Net().to(DEVICE)
MODEL_1.load_state_dict(torch.load("./Models/published_model_1.pt",weights_only=True))
MODEL_2 = Net().to(DEVICE)
MODEL_2.load_state_dict(torch.load("./Models/published_model_2.pt",weights_only=True))
MODEL_3 = Net().to(DEVICE)
MODEL_3.load_state_dict(torch.load("./Models/published_model_3.pt",weights_only=True))
MODEL_4 = Net().to(DEVICE)
MODEL_4.load_state_dict(torch.load("./Models/published_model_4.pt",weights_only=True))
MODEL_5 = Net().to(DEVICE)
MODEL_5.load_state_dict(torch.load("./Models/published_model_5.pt",weights_only=True))
MODEL_6 = Net().to(DEVICE)
MODEL_6.load_state_dict(torch.load("./Models/published_model_6.pt",weights_only=True))
MODEL_7 = Net().to(DEVICE)
MODEL_7.load_state_dict(torch.load("./Models/published_model_7.pt",weights_only=True))

for MODEL in [MODEL_1, MODEL_2, MODEL_3, MODEL_4, MODEL_5, MODEL_6, MODEL_7]:
    MODEL.eval();
    
x = input_for_model
x = x.type(torch.FloatTensor)
x = x.to(DEVICE)

In [14]:
new_columns = ['date'] + [f'forecast_{i}' for i in range(8)]
new_df = pd.DataFrame(columns=new_columns)

new_df['date'] = date_df['date']
for col in new_df.columns[1:]:
    new_df[col] = np.nan

result_df = new_df

In [15]:
for i in range(1,8):
    model = eval(f"MODEL_{i}")

    forecast_values = model(x)
    forecast_values = forecast_values.cpu().detach().numpy().flatten()

    result_df.loc[can_forecast_indices, f'forecast_{i}'] = forecast_values

In [16]:
result_df

,date,forecast_0,forecast_1,forecast_2,forecast_3,forecast_4,forecast_5,forecast_6,forecast_7
0,2021-06-11 14:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2021-06-11 15:00:00,NaN,387.453003,387.208496,432.113434,402.048035,410.836792,410.227264,407.636230
2,2021-06-12 15:00:00,NaN,333.751831,347.824615,388.722961,367.060760,387.073364,365.191986,345.038940
3,2021-06-12 16:00:00,NaN,335.369141,343.673065,392.021545,373.312653,386.911774,369.612701,348.237976
4,2021-06-12 17:00:00,NaN,335.952057,342.251831,393.836548,376.704559,387.444733,372.574219,347.072601
5,2021-06-13 16:00:00,NaN,338.898285,348.385803,370.291687,381.788483,370.776215,369.862488,336.146027
